# Preporcessing of raw `json` files

In [1]:
import re
import pandas as pd
import pickle

## Concating scraped `json` files

In [18]:
%%time
df = pd.DataFrame()
# loading 20 json files to dataframe
x = list(range(1, 21))
for i in x:
    file_path = f"./scraped-reviews/reviews{x[(i-1)]}.json"
    reviews = pd.read_json(file_path)
    df = pd.concat([df, reviews], sort=False)
    print(f"{file_path}  {reviews.shape[0]} reviews added, reviews total: {df.shape[0]}")

./scraped-reviews/reviews1.json  738665 reviews added, reviews total: 738665
./scraped-reviews/reviews2.json  351611 reviews added, reviews total: 1090276
./scraped-reviews/reviews3.json  461333 reviews added, reviews total: 1551609
./scraped-reviews/reviews4.json  408304 reviews added, reviews total: 1959913
./scraped-reviews/reviews5.json  380671 reviews added, reviews total: 2340584
./scraped-reviews/reviews6.json  333118 reviews added, reviews total: 2673702
./scraped-reviews/reviews7.json  351287 reviews added, reviews total: 3024989
./scraped-reviews/reviews8.json  472504 reviews added, reviews total: 3497493
./scraped-reviews/reviews9.json  271133 reviews added, reviews total: 3768626
./scraped-reviews/reviews10.json  338894 reviews added, reviews total: 4107520
./scraped-reviews/reviews11.json  498784 reviews added, reviews total: 4606304
./scraped-reviews/reviews12.json  291460 reviews added, reviews total: 4897764
./scraped-reviews/reviews13.json  445080 reviews added, review

In [19]:
%%time
# let's save our dataframe. Using pickle, because h5 was throwing error at me
df.to_pickle("steam_reviews.pkl")

CPU times: user 12.2 s, sys: 3.35 s, total: 15.6 s
Wall time: 24.8 s


## Creating review file

In [11]:
df = pd.read_pickle("steam_reviews.pkl")

In [21]:
df.head()

,product_id,page,page_order,recommended,date,text,hours,user_id,username,products,early_access,found_funny,compensation
0,646380.0,1.0,0,1.0,"27 December, 2017",Realistic simulation of a trip to and from a t...,0.7,7.656120e+16,turkuquantum,49.0,False,NaN,NaN
1,646380.0,1.0,1,1.0,"23 December, 2017","Starting the game is a pain, I think you have ...",0.2,NaN,FastLawyer,3569.0,False,1.0,NaN
2,646380.0,1.0,2,1.0,"8 December, 2017",Like casual skiing but don't have the possibil...,2.3,NaN,Cookie95,197.0,False,NaN,NaN
3,646380.0,1.0,3,1.0,"7 December, 2017",Very tough workout :D\nBought at a discount. T...,0.3,7.656120e+16,S0_GosU,79.0,False,NaN,NaN
4,646380.0,1.0,4,0.0,"23 July, 2017",I like the concept. I don't like that I stear ...,0.1,NaN,EntropyQ |FP|,115.0,False,NaN,NaN


In [22]:
# for a collaborative filter, we'll need only those columns
reviews = df[["product_id", "recommended", "date", "user_id", "username"]]

In [23]:
reviews.head()

,product_id,recommended,date,user_id,username
0,646380.0,1.0,"27 December, 2017",7.656120e+16,turkuquantum
1,646380.0,1.0,"23 December, 2017",NaN,FastLawyer
2,646380.0,1.0,"8 December, 2017",NaN,Cookie95
3,646380.0,1.0,"7 December, 2017",7.656120e+16,S0_GosU
4,646380.0,0.0,"23 July, 2017",NaN,EntropyQ |FP|


In [24]:
reviews.shape

(8158143, 5)

In [26]:
reviews.to_pickle("reviews.pkl")

## Exploring dataset

In [27]:
reviews = pd.read_pickle("reviews.pkl")

In [29]:
# how many reviews games have?
reviews["product_id"].value_counts().describe()

count     23863.000000
mean        341.873151
std        1969.881854
min           1.000000
25%           7.000000
50%          23.000000
75%         104.000000
max      117608.000000
Name: product_id, dtype: float64

In [28]:
# how many games each user had reviewed?
reviews["username"].value_counts().describe().apply(lambda x: format(x, "f"))

count    2978839.000000
mean           2.737019
std            9.868180
min            1.000000
25%            1.000000
50%            1.000000
75%            2.000000
max         3850.000000
Name: username, dtype: object

In [33]:
reviews_per_user = reviews["username"].value_counts()

In [34]:
reviews_per_user = reviews_per_user.to_frame()
reviews_per_user = reviews_per_user.reset_index()
reviews_per_user.columns = ["username", "reviews"]

In [35]:
reviews_per_user.head(10)

,username,reviews
0,123,3850
1,[deleted],3517
2,ForGamezCZ,2701
3,#,2649
4,Alex,1938
5,Epoc,1577
6,Chris,1492
7,Obey the Fist!,1470
8,Nick,1428
9,Salarn,1357


In [40]:
# let's see how many users have 20 or more reviews
reviews_per_user[reviews_per_user.iloc[:, 1] > 19].shape

(39614, 2)

In [42]:
# Let's see how many reviews we'll have in dataset, after droping top 100 (outliers, who writes 3000 reviews???)
# and keeping only users with 20 or more reviews
reviews_per_user["reviews"].iloc[100:36638].sum()

1735374

In [44]:
user_small = reviews_per_user.iloc[100:36638]

In [45]:
user_small.shape

(36538, 2)

In [46]:
user_small["reviews"].describe()

count    36538.000000
mean        47.495046
std         47.374179
min         21.000000
25%         25.000000
50%         32.000000
75%         49.000000
max        538.000000
Name: reviews, dtype: float64

In [47]:
user_small = user_small.reset_index(drop=True)

In [48]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None
):  # more options can be specified also
    print([user_small.iloc[:100]])

[               username  reviews
0                 Jesus      538
1                  Andy      537
2              Big Boss      527
3                  Sean      527
4      PhantonGualterio      525
5                 Snake      525
6                Frosty      520
7            Dick Rambo      520
8               Beridok      519
9    Tocăniță de jocuri      519
10                 Grim      518
11                Frost      518
12             bigh ◢ ◤      516
13               Wungle      515
14                 Will      515
15                  Leo      511
16                MGRza      510
17                 Dark      504
18                Cross      501
19                Stiff      500
20           ♿ Svinhugg      500
21                 Duck      498
22                  YCh      491
23                   :)      490
24             ᵂᴬᴸᴸᴱᴿ ❤      489
25                Jacob      486
26                Aaron      486
27                   󠀡󠀡      486
28                Jason      484
29       

In [57]:
# I see some wierd `blank` users

In [58]:
user_small.iloc[27,0]

'\U000e0021\U000e0021'

In [59]:
user_small.iloc[89,0]

'\U000e0021'

In [60]:
#they are using unicode signs, that don't render in jupyter

## Creating reviews files, with choosen set of users

In [62]:
user_small["reviews"].sum()

1735374

In [63]:
# let's create reviews files, but only with users from `user_small` dataframe

In [64]:
df = pd.merge(reviews, user_small, on="username")

In [65]:
df.head()

,product_id,recommended,date,user_id,username,reviews
0,646380.0,1.0,"7 December, 2017",7.656120e+16,S0_GosU,21
1,570.0,1.0,2018-07-09,7.656120e+16,S0_GosU,21
2,450390.0,1.0,2018-07-09,7.656120e+16,S0_GosU,21
3,368730.0,0.0,2017-03-25,7.656120e+16,S0_GosU,21
4,26800.0,1.0,2018-07-09,7.656120e+16,S0_GosU,21


In [67]:
user_small["reviews"].sum() == df.shape[0]

True

In [69]:
df = df.drop(["user_id",  "reviews"], axis=1)

In [70]:
df.shape[0]

1735374

In [71]:
# let's remove NaN rows from our dataframe
df = df.dropna()

In [72]:
df.shape[0]

1735173

In [73]:
df.to_hdf("reviews_small.h5", key="df", mode="w")